## Import Package

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Myskill/dataset/Online Retail Data.csv')
df.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN


## Data Preprocessing

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


In [ ]:
df.describe()

,quantity,price,customer_id
count,461773.000000,461773.000000,360853.000000
mean,9.088892,4.569963,15557.611138
std,86.366467,153.411566,1593.728741
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14210.000000
50%,3.000000,2.100000,15580.000000
75%,10.000000,4.210000,16938.000000
max,10200.000000,25111.090000,18287.000000


### Data Cleansing & Transformation

In [ ]:
# new dataframe
df_clean = df.copy()

In [ ]:
# missing value check
df.isnull().sum()

order_id             0
product_code         0
product_name      2718
quantity             0
order_date           0
price                0
customer_id     100920
dtype: int64

In [ ]:
# duplicated value check
df.duplicated().sum()

6479

In [ ]:
# convert order date into datetime
df_clean['order_date'] = df_clean['order_date'].astype('datetime64[ns]')

In [ ]:
# create new column date
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64[ns]')


In [ ]:
# turn product_name into lowercase
df_clean['product_name'] = df_clean['product_name'].str.lower()

In [ ]:
# create a column 'order_status' with the value 'cancelled' if the order_id starts with the letter 'c'
# and 'delivered' if the order_id does not start with the letter 'C'.
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')

In [ ]:
# transform customer_id into string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)

In [ ]:
# convert negative quantity values to positive
# negative values indicate that the order is cancelled
df_clean['quantity'] = df_clean['quantity'].abs()

In [ ]:
# replacing the product_name from the product_code that has multiple product_names with the most frequently appearing product_name
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')

### Missing value handling

In [ ]:
# percentage missing value, if <1% feel free to drop
data_missing_value = df_clean.isnull().sum().reset_index()
data_missing_value.columns = ['feature','missing_value']
data_missing_value['percentage'] = round((data_missing_value['missing_value']/len(df_clean))*100,3)
data_missing_value = data_missing_value.sort_values('percentage', ascending=False).reset_index(drop=True)
data_missing_value = data_missing_value[data_missing_value['percentage']>0]
data_missing_value

,feature,missing_value,percentage
0,product_name,388,0.084


In [ ]:
# handling missing value
missing_col = ['customer_id', 'product_name']
for col in missing_col:
  df_clean.dropna(subset=[col], inplace=True)
        #df_clean = df_clean[~df_clean['customer_id'].isna()] *alternative
        #df_clean = df_clean[~df_clean['product_name'].isna()] *alternative

In [ ]:
# delete row with none costumer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]

In [ ]:
# delete product_code or product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]

In [ ]:
# delete row with negative value in price
df_clean = df_clean[df_clean['price']>0]

In [ ]:
# create column 'amount = quantity * price'
df_clean['amount'] = df_clean['quantity'] * df_clean['price']

<ipython-input-21-90bd2909de6f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['amount'] = df_clean['quantity'] * df_clean['price']


In [ ]:
# delete outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,order_status,amount
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01-04,cancelled,4.25
1,493413,21724,panda and bunnies sticker sheet,1,2010-01-04 09:54:00,0.85,nan,2010-01-04,delivered,0.85
2,493413,84578,elephant toy with blue t-shirt,1,2010-01-04 09:54:00,3.75,nan,2010-01-04,delivered,3.75
3,493413,21723,alphabet hearts sticker sheet,1,2010-01-04 09:54:00,0.85,nan,2010-01-04,delivered,0.85
4,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,91.80


In [ ]:
# missing value check
data_missing_value = df_clean.isnull().sum().reset_index()
data_missing_value.columns = ['feature','missing_value']
data_missing_value

,feature,missing_value
0,order_id,0
1,product_code,0
2,product_name,0
3,quantity,0
4,order_date,0
5,price,0
6,customer_id,0
7,date,0
8,order_status,0
9,amount,0


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455496 entries, 0 to 455495
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      455496 non-null  object        
 1   product_code  455496 non-null  object        
 2   product_name  455496 non-null  object        
 3   quantity      455496 non-null  int64         
 4   order_date    455496 non-null  datetime64[ns]
 5   price         455496 non-null  float64       
 6   customer_id   455496 non-null  object        
 7   date          455496 non-null  datetime64[ns]
 8   order_status  455496 non-null  object        
 9   amount        455496 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 34.8+ MB


## RFM Segmentation

Rentensi : max_order_date (last purchase date) <br>
Frekuensi : order_cnt (total number purchase)<br>
Moniteri : total_order_value (average purchase amount)

In [ ]:
df_user = df_clean.groupby('customer_id', as_index=False).agg(order_cnt=('order_id','nunique'),max_order_date=('date','max'),total_order_value=('amount','sum'))
df_user

,customer_id,order_cnt,max_order_date,total_order_value
0,12346.0,5,2010-10-04,602.40
1,12608.0,1,2010-10-31,415.79
2,12745.0,2,2010-08-10,723.85
3,12746.0,2,2010-06-30,266.35
4,12747.0,19,2010-12-13,4094.79
...,...,...,...,...
3888,18284.0,2,2010-10-06,486.68
3889,18285.0,1,2010-02-17,427.00
3890,18286.0,2,2010-08-20,941.48
3891,18287.0,4,2010-11-22,2345.71


### Time elapsed between the last order and today"

In [ ]:
today = df_clean['date'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user.head()

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12346.0,5,2010-10-04,602.40,80
1,12608.0,1,2010-10-31,415.79,53
2,12745.0,2,2010-08-10,723.85,135
3,12746.0,2,2010-06-30,266.35,176
4,12747.0,19,2010-12-13,4094.79,10


In [ ]:
today

Timestamp('2010-12-23 00:00:00')

### RFM


In [ ]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)

In [ ]:
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=[0,
                                          np.percentile(df_user['order_cnt'], 20),
                                          np.percentile(df_user['order_cnt'], 40),
                                          np.percentile(df_user['order_cnt'], 60),
                                          np.percentile(df_user['order_cnt'], 80),
                                          df_user['order_cnt'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)

In [ ]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                    bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user.head()

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,monetary_score,frequency_score
0,12346.0,5,2010-10-04,602.40,80,2,3,4
1,12608.0,1,2010-10-31,415.79,53,3,2,1
2,12745.0,2,2010-08-10,723.85,135,2,3,2
3,12746.0,2,2010-06-30,266.35,176,1,2,2
4,12747.0,19,2010-12-13,4094.79,10,5,5,5


### Create Segmentation Column

In [ ]:
df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating']
)
df_user.head()

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,monetary_score,frequency_score,segment
0,12346.0,5,2010-10-04,602.40,80,2,3,4,08-At Risk
1,12608.0,1,2010-10-31,415.79,53,3,2,1,09-About to Sleep
2,12745.0,2,2010-08-10,723.85,135,2,3,2,10-Hibernating
3,12746.0,2,2010-06-30,266.35,176,1,2,2,10-Hibernating
4,12747.0,19,2010-12-13,4094.79,10,5,5,5,01-Champion


### RFM Summary using Pivot Table

In [ ]:
summary = pd.pivot_table(df_user, index='segment',
               values=['customer_id','day_since_last_order','order_cnt','total_order_value'],
               aggfunc={'customer_id': pd.Series.nunique,
                        'day_since_last_order': [np.mean, np.median],
                        'order_cnt': [np.mean, np.median],
                        'total_order_value': [np.mean, np.median]})
summary['pct_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)
summary

customer_id day_since_last_order         order_cnt  \
                           nunique                 mean median       mean   
segment                                                                     
01-Champion                    551            10.606171    9.0  18.535390   
02-Loyal Customers             546            40.807692   37.0   8.783883   
03-Potential Loyalists         523            23.548757   24.0   2.831740   
04-Can't Lose Them              64           122.859375  113.0  11.343750   
05-Need Attention              177            58.632768   59.0   3.406780   
06-New Customers                50            14.220000   16.0   1.000000   
07-Promising                   144            32.784722   34.0   1.000000   
08-At Risk                     427           140.545667  120.0   4.145199   
09-About to Sleep              352            58.767045   58.0   1.411932   
10-Hibernating                1059           196.632672  198.0   1.313503   

                              total_order_value           pct_unique  
                       median              mean    median             
segment                                                               
01-Champion              11.0       6936.138140  2777.140       14.2  
02-Loyal Customers        7.0       2636.771984  1966.000       14.0  
03-Potential Loyalists    3.0        771.385086   626.680       13.4  
04-Can't Lose Them        9.5       2876.881875  2268.405        1.6  
05-Need Attention         3.0        985.563960   824.190        4.5  
06-New Customers          1.0        244.689000   193.675        1.3  
07-Promising              1.0        286.665556   238.440        3.7  
08-At Risk                4.0       1156.649300   894.130       11.0  
09-About to Sleep         1.0        447.075284   334.755        9.0  
10-Hibernating            1.0        342.552146   256.850       27.2

In [ ]:
# User segementation count
segment_counts = df_user['segment'].value_counts()
print(segment_counts)

# Segment %
total_users = segment_counts.sum()
segment_percentage = (segment_counts / total_users) * 100
print("\nPersentase User Segmentation:")
print(segment_percentage)

segment
10-Hibernating            1059
01-Champion                551
02-Loyal Customers         546
03-Potential Loyalists     523
08-At Risk                 427
09-About to Sleep          352
05-Need Attention          177
07-Promising               144
04-Can't Lose Them          64
06-New Customers            50
Name: count, dtype: int64

Persentase User Segmentation:
segment
10-Hibernating            27.202671
01-Champion               14.153609
02-Loyal Customers        14.025173
03-Potential Loyalists    13.434369
08-At Risk                10.968405
09-About to Sleep          9.041870
05-Need Attention          4.546622
07-Promising               3.698947
04-Can't Lose Them         1.643976
06-New Customers           1.284357
Name: count, dtype: float64


In [ ]:
import plotly.express as px

segment_counts_sorted = segment_counts.sort_values(ascending=True)
fig = px.bar(segment_counts_sorted, orientation='h', title='User Segmentation Count',
             labels={'index': 'Segment', 'value': 'Count'})
fig.update_layout(
    width=700,
    height=400,
    title_x=0.5,
    title_y=0.95
)
fig.show()


## Insight
1. The "Hibernating" customer segment comprises the largest number of customers, totaling 1059 (27%), followed by "Champion" with 551 customers (14%), and "Loyal Customers" with 546 customers (14%).
2. The dominance of the "Hibernating" segment suggests a significant portion of customers are less active, requiring specific strategies to sustain their interest.
3. The "Potential Loyalists" segment, consisting of 523 customers (13%), shows potential for increasing transaction volume with tailored strategies.
4. Identification of the reasons behind the decline in activity for the "At Risk" (10%), "About to Sleep" (9%), and "Need Attention" (4%) segments is crucial to re-engage these customers.
5. The "New Customers" segment remains relatively small, indicating the need for product offerings to strengthen their loyalty.